In [35]:
# Day3 Project Challenge - COVID module

Find the original Notebook with detailed instructions by **Daniel Pass** [here](https://github.com/passdan/PracticalPythonProgrammingForBiologists/blob/main/Day4/P34B-Day4-Project-covid_modules.ipynb).

Find the input file [here](inputData/Day4-Project-nextclade.fasta) and [here](inputData/Day4-Project-nextclade_metadata.csv).

> In this exercise we are going to look at a complex dataset (n=126) of covid S protein reference sequences and associated metadata and statistics, and we're going to use some 3rd party modules to perform some higher-level analysis.
> 
> As we have seen today there are many (many!) ways to approach any dataset. First consider a question that you want to test from this data. If you are more interested in statistical testing then you can focus on the metadata and statistics with pandas/scipy. If you are more interested in sequence bioinformatics then you can work with the fasta data. However you'll likely need both to fully investigate.
>
> You have been provided with two files:
>
> * [<code>Day4-Project-nextclade_metadata.csv</code>](inputData/Day4-Project-nextclade_metadata.csv) - Data & Metadata of 126 reference covid19 strains and the Spike (S) protein modificiation data
> * [<code>Day4-Project-nextclade.fasta</code>](inputData/Day4-Project-nextclade.fasta) - Reference fasta sequence file for the corresponding spike proteins

SyntaxError: invalid syntax (2410099082.py, line 3)

In [1]:
import pandas as pd
from Bio import SeqIO, Seq, SeqRecord
import matplotlib.pyplot as plt
from scipy.stats import spearmanr
import inspect
import numpy as np

# Read in and filter the metadata file to remove bad and missing data
covid_data = pd.read_csv("inputData/Day4-Project-nextclade_metadata.csv")

covid_data_cleaned = covid_data.dropna()

# Read in the fasta sequence file. Only include sequences that have passed your step one filter!
covid_seqs = []

for seq in SeqIO.parse("inputData/Day4-Project-nextclade.fasta", "fasta"):

    if seq.id in list(covid_data_cleaned["seqName"]):
        covid_seqs.append(seq)

# A function to output the spike protein sequence and key statistics (GC%, length, etc)
def GCcalculator(ID):
    sequence = ID.seq

    GCcount = (sequence.count("G") + sequence.count("C")) / len(sequence) * 100

    return GCcount

for fasta_entry in covid_seqs:
  print(f"The sequence {fasta_entry.id} is {len(fasta_entry.seq)} nucleotides long and has GC content of {round(GCcalculator(fasta_entry))}%.")

# Calculate the mean number of substitutions per WHO clade
substs_per_clade = covid_data_cleaned.groupby("clade")["totalSubstitutions"].mean()

print(substs_per_clade)

# Calculate the number/size of gap regions (strings of Ns) - does this correspond with the quality assesment?
Ncount_list = []
qualityScore_list = []
qualityStatus_list = []
for fasta_entry in covid_seqs:
  # index = covid_data_cleaned.loc[covid_data_cleaned["seqName"] == fasta_entry.id, "index"]
  # print(f"{index=}")
  sequence = fasta_entry.seq
  Ncount = sequence.count('N')
  qualityScore = covid_data_cleaned.loc[covid_data_cleaned.seqName == fasta_entry.id, "qc.overallScore"].iloc[0]
  qualityStatus = covid_data_cleaned.loc[covid_data_cleaned.seqName == fasta_entry.id, "qc.overallStatus"].iloc[0]

  Ncount_list.append(Ncount)
  qualityScore_list.append(qualityScore)
  qualityStatus_list.append(qualityStatus)

stat, pvalue = spearmanr(Ncount_list, qualityScore_list)

print()
print("Spearman Rank test between NUMBER OF Ns and QUALITY SCORE")
print(f"R2 = {round(stat,3)} (p = {round(pvalue,5)})")
print()

plt.scatter(Ncount_list, qualityScore_list)
plt.xlabel("Number of Ns")
plt.ylabel("Overall quality score")
plt.show()

# Test for if binding efficiency correlates with any other variables, and plot the correlation.
def test_corr(series1, series2):
  stat, pvalue = spearmanr(series1, series2)
  arg_names = inspect.getfullargspec(test_corr).args

  if pvalue <= 0.05:
    test = "significant"
    return test, stat, pvalue
  else:
    test = "not significant"
    return test, stat, pvalue

for column in covid_data_cleaned:
  if column == "bindingEfficiency" or column == "index" or column.startswith("alignment"):
    #print(f"{column=} is not a valid column")
    continue

  column_list = list(covid_data_cleaned[column])

  if not all(isinstance(x, (float, int)) for x in column_list):
    continue

  bindingEfficiency_list = list(covid_data_cleaned.bindingEfficiency)
  test_results = test_corr(bindingEfficiency_list, column_list)
  
  print()
  print(f"Spearman Rank test between binding efficiency and {column} is {test_results[0].upper()}")
  print(f"R2 = {round(test_results[1],3)} (p = {round(test_results[2],5)})")
  print()

  m, b = np.polyfit(bindingEfficiency_list, column_list, 1)
  yp = np.polyval([m, b], bindingEfficiency_list)

  if not test_results[0].startswith("not"):
    plt.plot(bindingEfficiency_list, yp, "red")
  plt.scatter(bindingEfficiency_list, column_list)
  plt.xlabel("Binding efficiency")
  plt.ylabel(f"{column.capitalize()}")
  plt.show()

C:\Users\nicol\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\nicol\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


FileNotFoundError: [Errno 2] No such file or directory: '/content/Day4-Project-nextclade_metadata.csv'